In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import json
import pprint
import os
import clip
import imagehash
import cv2
import imagehash
from tqdm import tqdm_notebook as tqdm
from PIL import Image
import pickle
import shutil
import torch
import re
from string import punctuation
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download("stopwords")

stopwords_ = set(stopwords.words("english"))



[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("../data/val_data_map.csv")

In [3]:
df.head()

,caption,product,label
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566


In [4]:
df["step_1_lower"] = df["caption"].apply(lambda x:x.lower())
df.head()

,caption,product,label,step_1_lower
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...


In [5]:
df["step_2_no_sp_char"] = df["step_1_lower"].apply(lambda x:re.sub(r"[^A-Za-z0-9\s]+", "", x))
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...


In [6]:
def no_stop(x):
    tokens = word_tokenize(x)
    clean_tokens = [t for t in tokens if not t in stopwords_]
    clean_text = " ".join(clean_tokens)
    return clean_text
df["step_3_no_stop"] = df["step_2_no_sp_char"].apply(no_stop)
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...


In [7]:
df["step_4_no_num"] = df["step_3_no_stop"].apply(lambda x: re.sub(r"\b[0-9]+\b\s*", "", x))
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...


In [8]:
df["step_5_no_dupe"] = df["step_4_no_num"].apply(lambda x:" ".join(set(word_tokenize(x))))
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,float holding luohu baoan arrangement shenzhen...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,girls childrens students years children gift t...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,hou men suppository alternative sm sex qing ta...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,chinese burnin patron paintings lucky wall off...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,thick distressed straight lgnace brand slacksm...


In [9]:
def no_short_token(x):
    tokens = word_tokenize(x)
    clean_tokens = [t for t in tokens if len(t) > 1]
    clean_text = " ".join(clean_tokens)
    return clean_text
df["step_6_no_len_1_token"] = df["step_5_no_dupe"].apply(lambda x:" ".join(set(word_tokenize(x))))
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,float holding luohu baoan arrangement shenzhen...,float holding luohu baoan arrangement shenzhen...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,girls childrens students years children gift t...,girls childrens students years children gift t...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,hou men suppository alternative sm sex qing ta...,hou men women sm alternative sex qing tail fox...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,chinese burnin patron paintings lucky wall off...,boss chinese burnin patron lucky paintings wal...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,thick distressed straight lgnace brand slacksm...,thick distressed straight lgnace slacksmusic t...


In [10]:
def crop_token(x):
    tokens = x.split(" ")
    total_len = len(tokens)
    if total_len >3:
        percent_70 = round(total_len*0.7)
        crop_text = tokens[0:percent_70]
        return " ".join(crop_text)
    else:
        return " ".join(tokens)
df["caption_crop_first_70%"] = df["caption"].apply(crop_token)
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token,caption_crop_first_70%
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,float holding luohu baoan arrangement shenzhen...,float holding luohu baoan arrangement shenzhen...,Tanabata Valentine's Day Shenzhen Bao'an M Nan...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,girls childrens students years children gift t...,girls childrens students years children gift t...,Children's Toys Little Girl over 6 Years Old G...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,hou men suppository alternative sm sex qing ta...,hou men women sm alternative sex qing tail fox...,sm qing qu Alternative Sex between Men and Wom...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,chinese burnin patron paintings lucky wall off...,boss chinese burnin patron lucky paintings wal...,Traditional Chinese Painting Burnin' Up Yingke...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,thick distressed straight lgnace brand slacksm...,thick distressed straight lgnace slacksmusic t...,lgnace Lee Men's Jeans Thick Section Distresse...


In [11]:
def crop_token(x):
    tokens = x.split(" ")
    total_len = len(tokens)
    if total_len > 3:
        percent_70 = round(total_len*0.7)
        crop_text = tokens[-percent_70:]
        return " ".join(crop_text)
    else:
        return " ".join(tokens)
df["caption_crop_last_70%"] = df["caption"].apply(crop_token)
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token,caption_crop_first_70%,caption_crop_last_70%
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,float holding luohu baoan arrangement shenzhen...,float holding luohu baoan arrangement shenzhen...,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,M Nanshan Futian Luohu Car Decorative Flowers ...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,girls childrens students years children gift t...,girls childrens students years children gift t...,Children's Toys Little Girl over 6 Years Old G...,Old Girls 3-9 Boys 7 Students I Day Gift 10 Ch...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,hou men suppository alternative sm sex qing ta...,hou men women sm alternative sex qing tail fox...,sm qing qu Alternative Sex between Men and Wom...,between Men and Women with Metal Tail Fox Tail...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,chinese burnin patron paintings lucky wall off...,boss chinese burnin patron lucky paintings wal...,Traditional Chinese Painting Burnin' Up Yingke...,Yingkesong Landscape Painting Patron Living Ro...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,thick distressed straight lgnace brand slacksm...,thick distressed straight lgnace slacksmusic t...,lgnace Lee Men's Jeans Thick Section Distresse...,Section Distressed Straight Youth Autumn Tide ...


In [12]:
def crop_token(x):
    tokens = x.split(" ")
    total_len = len(tokens)
    if total_len >3:
        percent_70 = round(total_len*0.7)
        strt_idx = (total_len // 2) - (percent_70 // 2)
        end_idx = (total_len // 2) + (percent_70 // 2)
        crop_text = tokens[strt_idx:end_idx+1]
        return " ".join(crop_text)
    else:
        return " ".join(tokens)
df["caption_crop_middle_70%"] = df["caption"].apply(crop_token)
df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token,caption_crop_first_70%,caption_crop_last_70%,caption_crop_middle_70%
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,float holding luohu baoan arrangement shenzhen...,float holding luohu baoan arrangement shenzhen...,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,M Nanshan Futian Luohu Car Decorative Flowers ...,Shenzhen Bao'an M Nanshan Futian Luohu Car Dec...
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,girls childrens students years children gift t...,girls childrens students years children gift t...,Children's Toys Little Girl over 6 Years Old G...,Old Girls 3-9 Boys 7 Students I Day Gift 10 Ch...,over 6 Years Old Girls 3-9 Boys 7 Students I D...
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,hou men suppository alternative sm sex qing ta...,hou men women sm alternative sex qing tail fox...,sm qing qu Alternative Sex between Men and Wom...,between Men and Women with Metal Tail Fox Tail...,Alternative Sex between Men and Women with Met...
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,chinese burnin patron paintings lucky wall off...,boss chinese burnin patron lucky paintings wal...,Traditional Chinese Painting Burnin' Up Yingke...,Yingkesong Landscape Painting Patron Living Ro...,Burnin' Up Yingkesong Landscape Painting Patro...
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,thick distressed straight lgnace brand slacksm...,thick distressed straight lgnace slacksmusic t...,lgnace Lee Men's Jeans Thick Section Distresse...,Section Distressed Straight Youth Autumn Tide ...,Men's Jeans Thick Section Distressed Straight ...


In [13]:
df.to_csv("../data/val_data_prompt_clean.csv")